## 머신러닝 Pipeline & HyperParameter Tuning
- 이전 강의들에서 결측치 처리, 모델 학습 등을 진행하였다. 
- 이 때, 결측치 처리, 스케일링, 하이퍼 파라미터 등을 최소화하여 쉽게 연결할 수 있도록 도와준다. 
- 본 예제에서는 scikit-learn pipeline에 대해 학습할 예정이다. 

## 필수 라이브러리 불러오기
- 본 튜토리얼에 적합한 주요 라이브러리들을 불러온다. 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns 
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import set_config
import optuna

import xgboost
from xgboost import XGBRegressor

print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"seaborn version {sns.__version__}")
print(f"sklearn version {sklearn.__version__}")
print(f"optuna version {optuna.__version__}")
print(f"xgboost version {xgboost.__version__}")

numpy version 1.26.1
pandas version 2.1.1
seaborn version 0.13.0
sklearn version 1.3.1
optuna version 3.4.0
xgboost version 2.0.0


## 데이터 불러오기
- 본 실습을 위해 간단한 데이터를 불러온다. 

In [3]:
# DATA_PATH = "C:\\Users\\human\\Desktop\\mlops_tutorial\\data\\bike-sharing-demand\\"
DATA_PATH = "C:\\Users\\MSYUN\\Desktop\\ml_optuna_mlflow\\data\\bike-sharing-demand\\"
# DATA_PATH = "각자의 경로를 지정합니다."
train_df = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
submission = pd.read_csv(DATA_PATH + "sampleSubmission.csv")

train_df.shape, test.shape, submission.shape

((10886, 12), (6493, 9), (6493, 2))

## 데이터 전처리
- 데이터 전처리는 기존 강의와 비슷하게 진행한다. 
- 단, 범주형 데이터셋은 그대로 놔두도록 한다. 

In [4]:
# 타깃값 별도 저장, 결측치 제거
train_df = train_df[train_df['weather'] != 4]
y = train_df['count'] #타깃값

# count 컬럼 제거
train_df = train_df.drop(['count'], axis=1)

# 데이터 합치기
all_df = pd.concat([train_df, test])

# 날짜 데이터로 파생변수 만들기
all_df['date'] = pd.to_datetime(all_df['datetime'])
all_df['year'] = all_df['date'].dt.year
all_df['date'] = pd.to_datetime(all_df['datetime'])
all_df['year'] = all_df['date'].dt.year
all_df['month'] = all_df['date'].dt.month
all_df['day'] = all_df['date'].dt.day
all_df['hour'] = all_df['date'].dt.hour
all_df['weekday'] = all_df['date'].dt.day_name()

# 원-핫 인코딩을 위해 변경함
season_num = [1, 2, 3, 4]
season_str = ['Spring', 'Summer', 'Fall', 'Winter']
all_df['season'] = all_df['season'].replace(season_num, season_str)

weather_num = [1, 2, 3, 4]
weather_str = ['Clear', 'Few Clouds', 'Light Snow, Rain', 'Heavy Snow, Rain']
all_df['weather'] = all_df['weather'].replace(season_num, season_str)

del_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month', 'atemp']
all_df = all_df.drop(del_features, axis=1)

all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17378 entries, 0 to 6492
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      17378 non-null  object 
 1   holiday     17378 non-null  int64  
 2   workingday  17378 non-null  int64  
 3   weather     17378 non-null  object 
 4   temp        17378 non-null  float64
 5   humidity    17378 non-null  int64  
 6   year        17378 non-null  int32  
 7   day         17378 non-null  int32  
 8   hour        17378 non-null  int32  
 9   weekday     17378 non-null  object 
dtypes: float64(1), int32(3), int64(3), object(3)
memory usage: 1.3+ MB


## 데이터셋 분리

In [5]:
train = all_df.iloc[0:len(y), :]
test = all_df.iloc[len(y):, :]
train.shape, test.shape

((10885, 10), (6493, 10))

In [6]:
# train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.3, shuffle=True, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((7619, 10), (3266, 10), (7619,), (3266,))

## 데이터 전처리 파이프라인 만들기
- (1) 수치형 변수에 대해서는 StandardScaler() 클래스를 이용해서 표준화를 진행했다. 
- (2) 범주형 변수의 원핫인코딩하는 데이터 전처리 파이프라인을 만든다. 

In [7]:
# (1) 수치형 변수 StandardScaler(), PCA()
num_features = ['temp', 'humidity', 'hour']
num_transformer = Pipeline(
    steps = [("scaler", StandardScaler()), ("pca", PCA())]
)


# (2) 범주형 변수 원핫 인코딩 OneHotEncoder()
cat_features = ["season", "weather", "weekday"]
cat_transformer = OneHotEncoder(handle_unknown="ignore")


# (3) ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ("num", num_transformer, num_features), 
        ("cat", cat_transformer, cat_features)
    ]
)

# (4) 모델 정의
pipe_reg_model = Pipeline(
    steps = [("preprocessor", preprocessor), ("xgbrg", XGBRegressor())]
)

# 추가

## Grid Search
- 그리드 서치

In [8]:
set_config(display = 'diagram')

# (5) 그리드 서치
param_grid = {
    'xgbrg__max_depth' : [2, 3, 5, 7, 10],
    'xgbrg__n_estimators' : [10, 100, 500]
}

grid_pipe_model = GridSearchCV(pipe_reg_model, cv = 2, param_grid = param_grid)
log_y = np.log(y_train)
grid_pipe_model.fit(X_train, log_y)

C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\sit

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler()),
                                                                                         ('pca',
                                                                                          PCA())]),
                                                                         ['temp',
                                                                          'humidity',
                                                                          'hour']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['season',
                                                                          'weather',
                                                                          'weekday'])])),
                                       ('xgbrg',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_byleve...
                                                     learning_rate=None,
                                                     max_bin=None,
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'xgbrg__max_depth': [2, 3, 5, 7, 10],
                         'xgbrg__n_estimators': [10, 100, 500]})

## 평가지표 구현

In [9]:
def rmsle(y, y_,convertExp=True):
    
    # 지수변환
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
        
    # 로그변환 후 결측값을 0으로 변환 
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    
    # RMSLE 계산
    return np.sqrt(np.mean(calc))

In [10]:
# 예측
preds = grid_pipe_model.predict(X_valid)
log_y_val = np.log(y_valid)

# 선형 회귀 값
print("RMSLE 값 : ",rmsle(log_y_val, preds, True))

RMSLE 값 :  0.508652556236073


In [11]:
preprocessor.fit_transform(test)
final_preds = grid_pipe_model.predict(test)
submission['count'] = np.exp(final_preds)
submission.to_csv(DATA_PATH + 'submission.csv', index=False)

In [12]:
# 

## Random Search

In [14]:
# 
from scipy.stats import uniform
set_config(display = 'diagram')

# (6) Random Search
param_grid = {
    'xgbrg__max_depth' : np.arange(2, 10),
    'xgbrg__n_estimators': np.arange(10, 500),
    'xgbrg__eta': uniform(0.0001, 0.1),
}

randCV = RandomizedSearchCV(pipe_reg_model, cv = 2, param_distributions = param_grid)
log_y = np.log(y_train)
randCV.fit(X_train, log_y)

C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\sit

RandomizedSearchCV(cv=2,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                StandardScaler()),
                                                                                               ('pca',
                                                                                                PCA())]),
                                                                               ['temp',
                                                                                'humidity',
                                                                                'hour']),
                                                                              ('cat',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['season',
                                                                                'weather',
                                                                                'weekday'])])),
                                             ('xgbrg',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_...
       413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425,
       426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438,
       439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451,
       452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464,
       465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477,
       478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490,
       491, 492, 493, 494, 495, 496, 497, 498, 499])})

In [14]:
# 
preds = randCV.predict(X_valid)
log_y_val = np.log(y_valid)

# 선형회귀 값
print("RMSLE 값: ", rmsle(log_y_val, preds, True))

RMSLE 값 : 2.643472579381856


In [16]:
# 
preprocessor.fit_transform(test)
final_preds = randCV.predict(test)
submission['count'] = np.exp(final_preds)
submission.to_csv(DATA_PATH + 'submission.csv', index=False)

## 다중 모형 적용
- XGBoost외에 다중 모형을 적용해본다. 

In [17]:
# (1) 수치형 변수 StandardScaler()
num_features = ["temp", "humidity", "hour"]
num_transformer = Pipeline(
    steps = [("scaler", StandardScaler()), ("pca", PCA())]
)

# (2) 범주형 변수 원핫 인코딩 OneHotEncoder()
cat_features = ["season", "weather", "weekday"]
cat_transformer = OneHotEncoder(handle_unknown="ignore")

# (3) ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ("num", num_transformer, num_features), 
        ("cat", cat_transformer, cat_features)
    ]
)

- 다른 모형을 가져온다.

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

log_y_val = np.log(y_valid)
log_y_train = np.log(y_train)

regressors = [
    RandomForestRegressor()
   , DecisionTreeRegressor()
   , LinearRegression()
   , XGBRegressor()
]

# regressors = [pipe_rf, pipe_dt]
for regressor in regressors:
    pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor', regressor)
           ])
    model = pipeline.fit(X_train, log_y_train)
    preds = model.predict(X_valid)
    print(regressor)
    # 선형회귀 값
    print("RMSLE 값 :", rmsle(log_y_val, preds, True))

RandomForestRegressor()
RMSLE 값 : 0.49973556233535316
DecisionTreeRegressor()
RMSLE 값 : 0.7191077868503926
LinearRegression()
RMSLE 값 : 1.0366378762065063


C:\Users\MSYUN\Desktop\ml_optuna_mlflow\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)
RMSLE 값 : 0.5465131037364217


- 이번에는 Grid Search 및 Pipeline을 통해서 가장 좋은 Regressor를 작성해본다. 

In [19]:
log_y_val = np.log(y_valid)
log_y_train = np.log(y_train)

regressors = [
    RandomForestRegressor()
   , DecisionTreeRegressor()
   , XGBRegressor()
]

param_grids = [
    {
     'regressor__max_depth' : np.arange(2, 10)
    }, 
    {
     'regressor__max_depth' : np.arange(2, 10)
    }, 
    {
     'regressor__max_depth' : np.arange(2, 10), 
     'regressor__n_estimators' : np.arange(2, 10), 
     'regressor__eta': [0.0001, 0.1]
    }
]

for (regressor, param_grid) in zip(regressors, param_grids):
    print(regressor, param_grid)
    pipe_lr_model = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor',regressor)
           ])
    
    multi_gridCV = GridSearchCV(pipe_lr_model, cv=3, param_grid=param_grid)
    model = multi_gridCV.fit(X_train, log_y_train)
    preds = model.predict(X_valid)
    final_regressor = multi_gridCV.best_estimator_
    print(f"\n\nBEST CLASSIFIER: {final_regressor}")
    print("RMSLE 값 :", rmsle(log_y_val, preds, True))

RandomForestRegressor() {'regressor__max_depth': array([2, 3, 4, 5, 6, 7, 8, 9])}


BEST CLASSIFIER: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA())]),
                                                  ['temp', 'humidity', 'hour']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['season', 'weather',
                                                   'weekday'])])),
                ('regressor', RandomForestRegressor(max_depth=9))])
RMSLE 값 : 0.6178586203320343
DecisionTreeRegressor() {'regre